In [ ]:
using Distances, LinearAlgebra, Random, Statistics, Distributions, DelimitedFiles
include("KernelQuantile.jl")
include("KernelUtils.jl")
include("KernelLogistic.jl")
include("KernelMultinomial.jl")

In [ ]:
using CSV, DataFrames
df = CSV.read("codon_usage.csv", DataFrame)

# Replace "-" with missing and convert to Float64 for columns 7 and 8
for col in [7, 8]
    df[!, col] = [x == "-" ? missing : (isa(x, AbstractString) ? parse(Float64, x) : x) for x in df[!, col]]
end

X = Matrix(df[:,7:69])

In [ ]:
# 1. Impute missing values with column means
X_imputed = Matrix{Float64}(undef, size(X))
for j in 1:size(X, 2)
    col = X[:, j]
    non_missing = col[.!ismissing.(col)]
    μ = isempty(non_missing) ? 0.0 : mean(non_missing)
    X_imputed[:, j] = coalesce.(col, μ)
end

# 2. Standardize the imputed matrix (center + scale)
X_standardized = similar(X_imputed)
for j in 1:size(X_imputed, 2)
    col = X_imputed[:, j]
    μ = mean(col)
    σ = std(col)
    X_standardized[:, j] = iszero(σ) ? zeros(length(col)) : (col .- μ) ./ σ
end

In [ ]:
using StatsBase

# Step 1: Get unique sorted categories
categories = sort(unique(df[:,1]))

# Step 2: Create binary matrix
Y = zeros(size(df, 1), length(categories))
for (i, cat) in enumerate(df[:,1])
    col_index = findfirst(isequal(cat), categories)
    Y[i, col_index] = 1
end

In [ ]:
function nearest_label_classifier(X_train, Y_train, X_test)
    labels_train = onehot_to_label(Y_train)
    preds = similar(labels_train, size(X_test, 1))

    for i in 1:size(X_test, 1)
        # Compute squared Euclidean distances to all training points
        dists = sum((X_train .- X_test[i, :]') .^ 2, dims=2)
        nearest_idx = argmin(dists)
        preds[i] = labels_train[nearest_idx]
    end

    return preds
end

function onehot_to_label(Y::Matrix)
    return findmax.(eachrow(Y)) .|> x -> x[2]
end

function generate_nystrom_G(n::Int, m::Int, Y::Matrix; rng=Random.GLOBAL_RNG)
    # Validate input dimensions
    size(Y, 1) == n || error("Y must have $n rows")
    
    # Convert one-hot matrix to class labels
    y = [findfirst(==(1), row) for row in eachrow(Y)]
    any(isnothing, y) && error("Each row in Y must contain exactly one 1")
    y = convert(Vector{Int}, y)
    
    # Calculate class distribution and sampling targets
    classes = unique(y)
    class_counts = [count(==(c), y) for c in classes]
    class_proportions = class_counts ./ n
    
    # Allocate samples per class (maintain proportions)
    samples_per_class = zeros(Int, length(classes))
    for (i, prop) in enumerate(class_proportions)
        samples_per_class[i] = floor(Int, prop * m)
    end
    
    # Distribute remaining samples to largest fractional parts
    remainder = m - sum(samples_per_class)
    if remainder > 0
        fractional_parts = class_proportions .* m .- samples_per_class
        top_classes = partialsortperm(fractional_parts, 1:remainder; rev=true)
        samples_per_class[top_classes] .+= 1
    end

    # Perform stratified sampling
    selected_indices = Int[]
    for (i, c) in enumerate(classes)
        class_indices = findall(==(c), y)
        n_samples = min(samples_per_class[i], length(class_indices))
        append!(selected_indices, sample(rng, class_indices, n_samples; replace=false))
    end
    
    # Final shuffle to avoid class grouping
    shuffle!(rng, selected_indices)
    
    # Construct output matrix efficiently
    G = zeros(length(selected_indices), n)
    for (i, idx) in enumerate(selected_indices)
        G[i, idx] = 1.0
    end
    return G
end

In [ ]:
nreps = 20
n = size(X,1)*0.8
ms = [Int(floor(n/64)),Int(floor(n/32)),Int(floor(n/16)),Int(floor(n/8)),Int(floor(n/4))]
times_reduced = zeros(nreps,length(ms))
times_full = zeros(nreps,length(ms))
times_newton = zeros(nreps,length(ms))
times_linear = zeros(nreps)
times_nn = zeros(nreps)
ll_reduced = zeros(nreps,length(ms))
ll_full = zeros(nreps,length(ms))
ll_newton = zeros(nreps,length(ms))
ll_linear = zeros(nreps)
acc_reduced = zeros(nreps,length(ms))
acc_full = zeros(nreps,length(ms))
acc_newton = zeros(nreps,length(ms))
acc_linear = zeros(nreps)
acc_nn = zeros(nreps)
λs = exp10.(range(1, -5, length=30))

In [ ]:
for i in 1:nreps
    # Calculate split indices
    shuffled_indices = randperm(size(X,1))

    train_end = Int(floor(0.7 * size(X,1)))        # 70% training
    val_end = train_end + Int(floor(0.1 * size(X,1)))  # 10% validation

    # Training set (60%)
    train_idx = shuffled_indices[1:train_end]
    X_train = X_standardized[train_idx, :]
    Y_train = Y[train_idx, :]

    # Validation set (20%)
    println(train_end)
    println(val_end)
    val_idx = shuffled_indices[train_end+1:val_end]
    
    X_val = X_standardized[val_idx, :]
    Y_val = Y[val_idx, :]

    # Test set (20%)
    test_idx = shuffled_indices[val_end+1:end]
    X_test = X_standardized[test_idx, :]
    Y_test = Y[test_idx, :]

    X_train_val = [X_train;X_val]
    Y_train_val = [Y_train;Y_val]
    
    P_pred = zeros(size(Y_test,1),size(Y_test,2))
    row_sums_buffer = zeros(size(X_test,1))
    time1 = @elapsed B_linear,_,_ = FastMultinomial_reduced([ones(size(X_train,1)) X_train],Y_train, max_iters=5000)
    times_linear[i] = time1
    compute_probs_reduced!(P_pred,[ones(size(X_test,1)) X_test]*B_linear, row_sums_buffer)
    ll_linear[i] = loglikelihood(Y_test, P_pred)
    y_true = [argmax(Y_test[i,:]) for i in 1:size(X_test,1)]
    y_pred = [argmax(P_pred[i,:]) for i in 1:size(X_test,1)]
    acc_linear[i] = mean(y_true .== y_pred)
    time2 = @elapsed preds_test = nearest_label_classifier(X_train_val, Y_train_val, X_test)
    labels_test = onehot_to_label(Y_test)

        # Compute accuracy
    acc_nn[i] = mean(preds_test .== labels_test)
    println("Test accuracy: ", round(acc_nn[i]* 100, digits=2), "%")
    times_nn[i] = time2
    σ = 10.0
    K = rbf_kernel(X_train,σ)
    n = size(X_train,1)
    for j in 1:length(ms)
        m = ms[j]
        G = generate_nystrom_G(n,m,Y_train)
        @time B1s, total_time1 = KernelMultinomial(Y_train,K,G,λs,ϵ=1e-4,verbose=false,tol=1e-2,max_iters=500)
        times_reduced[i,j] = total_time1
        best_metric = -Inf
        best_k = 1
        for k in 1:length(λs)
            compute_probs_reduced!(P_pred,predict_krr(X_train,X_val,G,B1s[k,:,:],σ), row_sums_buffer)
            if loglikelihood(Y_val, P_pred)>best_metric
                best_metric = loglikelihood(Y_val, P_pred)
                best_k = k
            end
        end
        K1 = rbf_kernel(X_train_val,σ)
        G1 = generate_nystrom_G(size(X_train_val,1),m,Y_train_val)
        best_B,_,_,_ = KernelMultinomial(Y_train_val,K1,G1,λs[best_k],ϵ=1e-4,verbose=false,tol=1e-2)
        compute_probs_reduced!(P_pred,predict_krr(X_train_val,X_test,G1,best_B,σ), row_sums_buffer)
        ll_reduced[i,j] = loglikelihood(Y_test, P_pred)
        y_true = [argmax(Y_test[i,:]) for i in 1:size(X_test,1)]
        y_pred = [argmax(P_pred[i,:]) for i in 1:size(X_test,1)]
        acc_reduced[i,j] = mean(y_true .== y_pred)
        @time B2s, total_time2 = KernelMultinomial_full(Y_train,K,G,λs,ϵ=1e-4,verbose=false,tol=1e-2,max_iters=500)
        times_full[i,j] = total_time2
        best_metric = -Inf
        best_k = 1
        for k in 1:length(λs)
            compute_probs!(P_pred,predict_krr(X_train,X_val,G,B2s[k,:,:],σ), row_sums_buffer)
            if loglikelihood(Y_val, P_pred)>best_metric
                best_metric = loglikelihood(Y_val, P_pred)
                best_k = k
            end
        end
        best_B,_,_,_ = KernelMultinomial_full(Y_train_val,K1,G1,λs[best_k],ϵ=1e-4,verbose=false,tol=1e-2)
        compute_probs!(P_pred,predict_krr(X_train_val,X_test,G1,best_B,σ), row_sums_buffer)
        ll_full[i,j] = loglikelihood(Y_test, P_pred)
        y_true = [argmax(Y_test[i,:]) for i in 1:size(X_test,1)]
        y_pred = [argmax(P_pred[i,:]) for i in 1:size(X_test,1)]
        acc_full[i,j] = mean(y_true .== y_pred)
        if m>1000
            continue
        else
            @time B3s, total_time3 = KernelMultinomial_newton(Y_train,K,G,λs,ϵ=1e-4,verbose=false,tol=1e-2)
            times_newton[i,j] = total_time3
            best_metric = -Inf
            best_k = 1
            for k in 1:length(λs)
                compute_probs_reduced!(P_pred,predict_krr(X_train,X_val,G,B3s[k,:,:],σ), row_sums_buffer)
                if loglikelihood(Y_val, P_pred)>best_metric
                    best_metric = loglikelihood(Y_val, P_pred)
                    best_k = k
                end
            end
            best_B,_,_,_ = KernelMultinomial_newton(Y_train_val,K1,G1,λs[best_k],ϵ=1e-4,verbose=false,tol=1e-2)
            compute_probs_reduced!(P_pred,predict_krr(X_train_val,X_test,G1,best_B,σ), row_sums_buffer)
            ll_newton[i,j] = loglikelihood(Y_test, P_pred)
            y_true = [argmax(Y_test[i,:]) for i in 1:size(X_test,1)]
            y_pred = [argmax(P_pred[i,:]) for i in 1:size(X_test,1)]
            acc_newton[i,j] = mean(y_true .== y_pred)
        end
        writedlm("times_reduced.csv",times_reduced,",")
        writedlm("times_full.csv",times_full,",")
        writedlm("times_newton.csv",times_newton,",")
        writedlm("times_linear.csv",times_linear,",")
        writedlm("times_nn.csv",times_nn,",")
        writedlm("ll_reduced.csv",ll_reduced,",")
        writedlm("ll_full.csv",ll_full,",")
        writedlm("ll_newton.csv",ll_newton,",")
        writedlm("ll_linear.csv",ll_linear,",")
        writedlm("acc_reduced.csv",acc_reduced,",")
        writedlm("acc_full.csv",acc_full,",")
        writedlm("acc_newton.csv",acc_newton,",")
        writedlm("acc_linear.csv",acc_linear,",")
        writedlm("acc_nn.csv",acc_nn,",")
    end
end

In [ ]:
mean(times_reduced,dims=1)

In [ ]:
std(times_reduced,dims=1)

In [ ]:
mean(times_full,dims=1)

In [ ]:
std(times_full,dims=1)

In [ ]:
mean(times_newton,dims=1)

In [ ]:
std(times_newton,dims=1)

In [ ]:
mean(ll_reduced,dims=1)

In [ ]:
std(ll_reduced,dims=1)

In [ ]:
mean(ll_full,dims=1)

In [ ]:
std(ll_full,dims=1)

In [ ]:
mean(ll_newton,dims=1)

In [ ]:
std(ll_newton,dims=1)

In [ ]:
mean(acc_reduced,dims=1)

In [ ]:
std(acc_reduced,dims=1)

In [ ]:
mean(acc_full,dims=1)

In [ ]:
std(acc_full,dims=1)

In [ ]:
mean(acc_newton,dims=1)

In [ ]:
std(acc_newton,dims=1)

In [ ]:
mean(acc_nn)

In [ ]:
std(acc_nn)